### tranformation des annotations au format pascalvoc vers yolo du dataset fruit_images_for_object_detection

In [20]:
import os
from matplotlib import pyplot as plt


In [ ]:
import xml.etree.ElementTree as ET

def get_label_id(dict,label):
    if label in dict.keys():
        return dict[label]
    else:
        dict[label] = len(dict)
        return dict[label]

def read_content(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []

    for boxes in root.iter('object'):

        labelname = root.find("object/name").text

        ymin, xmin, ymax, xmax = None, None, None, None

        ymin = int(boxes.find("bndbox/ymin").text)
        xmin = int(boxes.find("bndbox/xmin").text)
        ymax = int(boxes.find("bndbox/ymax").text)
        xmax = int(boxes.find("bndbox/xmax").text)
        img_width = int(root.find("size/width").text)
        img_height = int(root.find("size/height").text)
        if img_width ==0 or img_height ==0:
            im = plt.imread(xml_file.replace("xml","jpg"))
            img_height, img_width, _ = im.shape


        list_with_single_boxes =[labelname, xmin, ymin, xmax, ymax, img_width, img_height]
        list_with_all_boxes.append(list_with_single_boxes)

    return  list_with_all_boxes

def to_yolo(xcenter, ycenter, obj_w, obj_h, img_w, img_h):
    x = xcenter / img_w
    y = ycenter / img_h

    w = obj_w / img_w
    h = obj_h / img_h    

    mystring = str("0 " + str(round(x, 7)) + " " + str(round(y, 7)) + " " + str(round(w, 7)) + " " + str(round(h, 7)))
    
    return mystring

In [11]:
path = "/home/usr/code/pdufourny/Projet/fruit_images_for_object_detection/"
path_train = path+"train_zip/train/"
path_test = path+"test_zip/test/"

lst_file_input = [path_train+fname for fname in os.listdir(path_train) if fname.endswith(".xml")]
lst_labels = {}


In [19]:
for xml_file in lst_file_input:
    print(xml_file)
    list_with_all_boxes = read_content(xml_file)
    out_file = open(xml_file[:-4]+".txt", "w")
    for box in list_with_all_boxes:
        (label,x_min, y_min, x_max, y_max,img_width,img_height) = box
        label_num = get_label_id(lst_labels, label)
        box[0] = str(label_num)

        x_center = (x_max + x_min) / 2
        y_center = (y_max + y_min) / 2
    
        w =  x_max - x_min
        h = y_max - y_min
        yolo_str = to_yolo(x_center, y_center, w, h, img_width, img_height)
        out_file.write(yolo_str + "\n")
    out_file.close()

/home/usr/code/pdufourny/Projet/fruit_images_for_object_detection/train_zip/train/orange_46.xml
/home/usr/code/pdufourny/Projet/fruit_images_for_object_detection/train_zip/train/apple_14.xml
/home/usr/code/pdufourny/Projet/fruit_images_for_object_detection/train_zip/train/orange_20.xml
/home/usr/code/pdufourny/Projet/fruit_images_for_object_detection/train_zip/train/banana_71.xml


ZeroDivisionError: float division by zero